##  LangChain快速入门 (学习笔记)

https://techdiylife.github.io/blog/topic.html?category2=t07&blogid=0043

In [6]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma2", base_url="http://localhost:11434")

llm.invoke("你是谁？")

'我是一个大型语言模型，由 Google 训练。\n\n我可以生成文本、翻译语言、编写不同类型的创意内容，并尝试回答您的问题以尽我所能提供有用的信息。\n\n我仍在学习中，但我的目标是成为一个有用的工具，可以帮助人们理解世界并完成任务。'

In [10]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate

model = Ollama(model="qwen2")
prompt = ChatPromptTemplate.from_messages([
    ("system", "现在你的名字叫小王."),
    ("user", "{input}")
])

chain = prompt | model
chain.invoke("你是谁？ 用JSON格式输出")

'{\n  "name": "小王",\n  "role": "AI助手"\n}'

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# 定义提示模板
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 初始化 StrOutputParser
output_parser = StrOutputParser()

# 创建链
chain = prompt | model | output_parser

# 调用链并获取结果
result = chain.invoke({"context": "服务型制造研究院在一个科研机构", "question": "服务型制造研究院在地处什么城市?"})
print(result)

文中没有提到服务型制造研究院所在的具体城市，因此无法回答这个问题。


In [23]:
# Retrieval Chain 检索链条
#!pip install beautifulsoup4
#!pip install faiss-cpu

from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

# 第一步：初始化模型对象：LLM，Embedding
llm = Ollama(model="qwen2")
embeddings = OllamaEmbeddings(model="qwen2")
response1 = llm.invoke("自由主义女权是什么？？")
print(f"检索前：{response1}")


# 第二步：获取数据
loader = WebBaseLoader("https://plato.stanford.edu/entries/feminism-liberal/")
docs = loader.load()

# 第三步：文本拆分，文本Vector化
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


# 第四步：准备Prompt，创建文档处理的LLMChain，检索Chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain) #建立检索

# 第五步：代码执行与测试
response2 = document_chain.invoke({
    "input": "自由主义女权是什么？",
    "context": [Document(page_content="feminism-liberal")]
})
print(f"检索前（运行测试）：{response2}")

response3 = retrieval_chain.invoke({"input": "自由主义女权是什么？"})
print(f"检索后：{response3['answer']}")

检索前：自由主义女权（Liberal feminism）是女权运动的一个重要流派，它强调通过法律、政治和经济上的改革来实现性别平等。自由主义女权的核心理念在于，女性应该享有与男性相同的权利，包括受教育的权利、就业的机会、工资的公平性以及参与民主决策的过程。这个理论认为，通过改变制度和社会结构中的不平等待遇，可以逐步消除性别歧视。

自由主义女权主张：

1. **法律平等**：通过立法来确保男女在法律面前平等对待，比如废除针对女性的歧视性法律、推动男女同工同酬等。
2. **经济独立**：强调女性应该有获得良好教育和职业发展的机会，以提高其经济自立能力。
3. **政治参与**：鼓励女性参与政治和社会事务，增加她们在决策过程中的代表性和影响力。
4. **消除性别角色刻板印象**：倡导打破社会对男女角色的传统期望，促进性别平等的社会观念。

自由主义女权理论与激进女权、社会主义女权等其他女权派别有所不同。它更侧重于通过渐进式的变革来实现目标，并认为在现行的法律和制度框架内进行改革是可行且有效的途径。
检索前（运行测试）：自由主义女权（feminism-liberal）指的是基于自由主义思想的女权观点或运动。它强调个人自由、平等以及通过法律和制度来实现性别平等等核心原则。

自由主义女权通常主张以下几点：

1. 法律面前人人平等：认为男女应当在法律面前享有同等的权利，包括但不限于工作权利、教育机会、投票权和平等的薪资待遇。
2. 家庭与职场的自主选择：支持个体根据自己的意愿和能力在家庭生活和社会活动中做出自由决定。这可能涉及到反对传统的性别角色分配，并提倡父亲和母亲都有平等参与育儿的责任。
3. 政策改革以促进平等：倡导政策上的改变，如平等薪资、反性骚扰法、工作与生活平衡的措施等，以减少性别歧视并确保女性在社会各个领域享有公平的机会。
4. 个人选择权和自主权：强调个体对于自己的身体和生活方式有决策权，并支持合法的人权和自由权利，包括但不限于堕胎权、同性婚姻以及性别认同的权利。

总之，自由主义女权关注通过法律、政策和社会改革来促进男女之间的平等和保护个人的自由。它不依赖于特定的社会或文化环境中的传统角色分配，而是强调个体的自我实现和个人自由。
检索后：自由主义女权是指一种政治哲学观点，它将女性的权益和地位置于个人自由和民主制度的核心。在这一框架下，女权主义者强调法

In [24]:
#  基于历史对话记录的检索Chain

from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 第一步： 准备LLM，Embedding模型，读取数据，数据处理
llm = Ollama(model="qwen2")
llm.invoke("langsmith是做什么的？")
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

embeddings = OllamaEmbeddings(model="qwen2") #模型默认为llama2
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

retriever = vector.as_retriever()

# 第二步：创建检索Chain，通过LLM根据历史对话记录检索相关内容
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "根据上述对话，生成一个搜索查询以查找与对话相关的信息")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

# 动作确认：用来确认retriever_chain的执行情况，执行检索，不会真的回答问题"告诉我怎么做"
#chat_history = [HumanMessage(content="LangSmith的网址？"), AIMessage(content="可以！")]
#response1 = retriever_chain.invoke({
#    "chat_history": chat_history,
#    "input": "告诉我怎么做"
#})

# 第三步：创建检索chain，根据检索出的内容，历史对话，与用户的问题产生回答
prompt = ChatPromptTemplate.from_messages([
    ("system", "根据下面的上下文回答用户的问题：\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

# 第四步：代码执行与测试
chat_history = [HumanMessage(content="LangSmith能帮助测试我的LLM应用吗？"), AIMessage(content="可以！")]
response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "告诉我怎么做"
})
print(response["answer"])

在使用LangSmith进行LLM应用的测试时，可以遵循以下步骤来优化和监控你的应用程序：

### 1. **快速启动（Quick Start）**
- 访问LangSmith并注册一个帐户。
- 启用追踪功能以便跟踪应用程序的性能。

### 2. **用户指南（User Guide）**
- 阅读用户指南以了解如何使用平台的各项功能，包括测试、评估和生产监控等。

### 3. **Beta 测试（Beta Testing）**
- 在开发初期或Beta阶段启用beta测试模式。这能让你收集应用在真实场景下的数据，并观察其表现。
- 进行用户反馈收集，分析应用的性能并标注问题点。

### 4. **捕获反馈（Capturing Feedback）**
- 启动应用时，确保从用户那里获取有关生成响应的人类反馈。这有助于识别有趣和边缘情况的问题。
- 在LangSmith中为反馈设置评分，并使用特定的反馈标记过滤追踪，以便专注于低分反馈。

### 5. **标注追踪（Annotating Traces）**
- 使用注释功能对应用性能进行详细说明。这可以帮助你捕获关键数据点、构建基准测试集以及在需要时进行人工标注。
- 安排内部或外部专家作为注释者，以帮助标记不同的应用版本并比较它们的性能。

### 6. **添加到数据集（Adding Runs to a Dataset）**
- 随着应用程序进入Beta阶段和生产阶段，请继续收集数据以优化其性能。将运行结果添加到数据集中，以便于在实际场景下进行广泛的测试覆盖。
- 利用LangSmith的自动评分、注释队列和数据集功能来处理大规模数据并进行比较。

### 7. **生产监控与自动化（Production Monitoring & Automations）**
- 监控应用程序性能的关键指标（如延迟、成本和反馈分数），确保在大量用户中提供理想的结果。
- 集成实时评估和自动化流程，用于在生产环境中快速处理和评分追踪。
- 利用线程功能来整理单一对话中的追踪，以便跟踪应用的多轮交互。

### 8. **深入调试（Debugging）**
- 使用LangSmith提供的调试信息，分析和识别序列中可能出现的问题点。这可以帮助你理解模型的运行方式并快速解决问题。

通过遵循这些步骤和利用LangSmith的特

In [26]:
# Agent 智能体

from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# 第一步：初始化模型（模型建议使用GPT3.5，GPT4等大模型，小模型难以胜任）
OPENAI_API_KEY="sk-...." # 为避免泄露，可以设置到环境变量
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)
embeddings = OllamaEmbeddings(model="qwen2")

# 第二步：数据处理，embedding等准备
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 第三步：创建工具（检索工具）
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
tools = [retriever_tool] #原代码中，还有一个检索天气预报的工具


# 第四步：设置Prompt，创建基于LLM的Agent，以及AgentExecutor来执行和管理Agent。
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt) #创建Agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 第五步：代码执行与测试
## 无历史对话的问答
response1 = agent_executor.invoke({"input": "如何使用 langsmith 来做测试？"})
print(response1)

## 有历史对话的问答
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
response2 = agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
print(response2)

ImportError: cannot import name 'version_short' from 'pydantic.version' (d:\repo\LLM-complete\.conda\Lib\site-packages\pydantic\version.cp311-win_amd64.pyd)